In [ ]:
import gc
import os
import psutil
import sys

import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [ ]:
# Define all constants
_bytes_in_mb = 1048576

In [ ]:
def get_process_stats(pid=0, seconds=1, cores=False):
    # This function is used to get current running process's info like cpu/memory
    # It should only be called periodically at the process/node level, not pipeline specific

    if pid <= 0:
        pid = os.getpid()
    process = psutil.Process(pid)
    info = 'Current running process (pid %s) details: CPU: %.2f' % (pid, process.cpu_percent(seconds))
    if cores:
        info += '\nNo of physical cores in system: %d logical cores: %d' % (psutil.cpu_count(logical=False), psutil.cpu_count())

    info += '\nUsed Memory: %.2f MB %s' % (process.memory_info().rss / _bytes_in_mb, process.memory_info())
    return info

def get_object_size(input_obj):
    # Measure the Real Size of Any Python Object
    # https://towardsdatascience.com/the-strange-size-of-python-objects-in-memory-ce87bdfbb97f
    memory_size = 0
    ids = set()
    objects = [input_obj]

    while objects:
        new = []
        for obj in objects:
            if id(obj) not in ids:
                ids.add(id(obj))
                memory_size += sys.getsizeof(obj)
                new.append(obj)
        objects = gc.get_referents(*new)
    return memory_size

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print('%d GPUs available. Enabling memory_growth', len(physical_devices))
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print('NO GPUs available.')
physical_devices = tf.config.list_physical_devices('CPU')
print('%d CPUs available.' % len(physical_devices))
print('get_process_stats: %s' % get_process_stats(cores=True))

### Example data:  
    - An experimental drug was tested on individuals from ages 13 to 100 in a clinical trial.
    - The trial had 2000 participants. half were under 65 years old, half were 65 years or older.
    - Around 95% of patients 65 or older experienced side effects.
    - Around 95% of patients under 65 experienced NO side effects.

In [ ]:
# This function generates a list of training samples and labels
def generate_random_samples_labels(n=2000):
    samples = []
    labels  = []
    for i in range(int(n*0.025)):
        # 5% of younger individuals who did experience side effects
        samples.append(randint(13, 64))
        labels.append(1)
        # 5% of older individuals who did NOT experience side effects
        samples.append(randint(65, 100))
        labels.append(0)

    for i in range(int(n*0.475)):
        # 95% of younger individuals who did NOT experience side effects
        samples.append(randint(13, 64))
        labels.append(0)
        # 95% of older individuals who did experience side effects
        samples.append(randint(65, 100))
        labels.append(1)

    return samples, labels

In [ ]:
train_samples, train_labels = generate_random_samples_labels()
len(train_labels), len(train_samples)

In [ ]:
list(zip(train_samples, train_labels))[:8]

In [ ]:
# Transform this data to the format expected by model.fit
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
# Shuffle two lists to get rid of any imposed order in data generation process
train_labels, train_samples = shuffle(train_labels, train_samples)
# sklearn.utils.shuffle make sure their shuffle orders are consistent (in the same order)

In [ ]:
list(zip(train_samples, train_labels))[:8]

In [ ]:
# Normalize the data to make training of neural networks quicker and more efficient
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))
# reshape(-1,1) -1=unknown : reshape 1D into 2D with size of each row to 1

In [ ]:
list(zip(scaled_train_samples, train_labels))[:5]

## Simple tf.keras Sequential model

In [ ]:
# Sequential model is a linear stack of layers
model = Sequential([
    # second (or first hidden) layer needs input_shape
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# shuffle=True by default, split occurs before the training set has been shuffled. It just takes last 10% of training data
model.fit(x=scaled_train_samples, y=train_labels, validation_split=0.1, batch_size=10, epochs=30, shuffle=True, verbose=2)

### Prediction

In [ ]:
test_samples, test_labels = generate_random_samples_labels(200)
test_samples, test_labels = np.array(test_samples), np.array(test_labels)
test_labels, test_samples = shuffle(test_labels, test_samples)
scaler = MinMaxScaler(feature_range=(0,1))
scaled_test_samples = scaler.fit_transform(test_samples.reshape(-1,1))
scaled_test_samples[0]

In [ ]:
# there is NO output during prediction even if we set verbose=2
predictions = model.predict(x=scaled_test_samples, batch_size=10, verbose=0)
predictions

In [ ]:
rounded_preds = np.argmax(predictions, axis=-1)
rounded_preds

### Confusion Matrix

In [ ]:
%matplotlib inline
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# https://deeplizard.com/learn/video/km7pxKy4UHU
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm = confusion_matrix(y_true=test_labels, y_pred=rounded_preds)

In [ ]:
cm_plot_labels = ['NO_side_effects', 'had_side_effects']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

### Saving & load a model

#### This model.save function saves:  
- The architecture of the model, allowing to recreate the model
- The weights of the model
- The training configuration (loss, optimizer etc..)
- The state of the optimizer, allowing to resume training exactly where you left off

In [ ]:
import os.path
file_path = '../models/medical_trial_model.h5'
if not os.path.isfile(file_path):
    model.save(file_path)

In [ ]:
from tensorflow.keras.models import load_model
new_model = load_model(file_path)
new_model.summary()

In [ ]:
new_model.get_weights()

In [ ]:
new_model.optimizer

#### 2. model.to_json()  
If you only need to save the architecture of a model, and not its weights or training configuration  
you can use the following function to save the architecture only

In [ ]:
model_json = model.to_json()
model_json